In [1]:
dbutils.library.installPyPI('opencv-python')
dbutils.library.restartPython()

In [2]:
import csv
import json
import sys
import binascii
import numpy as np
import datetime
import pandas as pd
from collections import Counter
import os, errno
from os import path
import cv2

In [3]:
#Path to Tab Delimited File of Image + Tags
SOURCE_DATA = '/mnt/RAW/FILES/SYNAPSE/POC/MS-TAGGED-DATA/MarsData-1-new2.tsv'
IMAGE_OUTPUT_DIR_LAKE = '/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/'
IMAGE_OUTPUT_DIR = '/local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/'


In [4]:
%sh
rm /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER_DATA.ZIP
ls /local_disk0/tmp/ -lth

total 640K
drwxr-xr-x 126 root root 4.0K Feb 23 18:42 PRODUCT_IMAGES_CLASSIFIER
-rw-r--r-- 1 root root 59K Feb 23 18:08 liblz4-java3733098620950031648.so
-rw-r--r-- 1 root root 516K Feb 23 18:08 libzstd-jni6126653171255846944.so
drwxr-xr-x 2 root root 4.0K Feb 23 18:08 libIsolationDir-860175fb-141a-49cf-93b6-e3fe329183a5
drwx------ 4 root root 4.0K Feb 23 18:08 root
drwxr-xr-x 3 root root 4.0K Feb 23 18:08 1582481285396-0
drwxr-xr-x 2 root root 4.0K Feb 23 18:07 clusterWideLibDir
-rw-r--r-- 1 root root 5.5K Feb 23 18:06 _CleanRShell.r8449128554578305797resource.r
-rw-r--r-- 1 root root 6.4K Feb 23 18:06 _dbutils.r6561986766414834734resource.r
-rw-r--r-- 1 root root 301 Feb 23 18:06 _rServeScript.r7469834063230077324resource.r
-rw-r--r-- 1 root root 1.5K Feb 23 18:06 _startR.sh2892367215770080401resource.r
drwx------ 2 root root 4.0K Feb 23 18:06 468fdf20-4e49-451f-a4d7-e50e5dae17cb_resources
drwxr-xr-x 2 root root 4.0K Feb 23 18:05 spark-5ad85925-9a58-4e16-8cb3-f9790c63f21e
drwxr-xr-x 2 root root 4.0K Feb 23 18:05 spark-537e041e-540a-4c6f-9520-f9ab9d3a6f45
drwxr-xr-x 2 root root 4.0K Feb 23 18:05 spark-2a011c77-5175-485d-bc70-3874338f409c
drwxr-xr-x 3 root root 4.0K Feb 23 18:05 repl

In [5]:
#Remove Existing Faces Content
#For Datalake:
#dbutils.fs.rm(IMAGE_OUTPUT_DIR, True)
#dbutils.fs.mkdirs(IMAGE_OUTPUT_DIR)

#For Local Storage:
import shutil
if os.path.exists(IMAGE_OUTPUT_DIR):
  shutil.rmtree(IMAGE_OUTPUT_DIR)
os.mkdir(IMAGE_OUTPUT_DIR)

In [6]:
#Open TSV with Data Frame
df_tagged_data = spark.read.format('csv').options(header='false', inferSchema='false', delimiter='\t').load(SOURCE_DATA)
df_tagged_data = df_tagged_data.toPandas()

In [7]:
#Create a Small Slice of 10 Rows for Testing
df_tagged_data_10 = df_tagged_data[:100]

In [8]:
tag_counter = dict()
item_counter = dict()
image_counter = dict()

In [9]:
#ACTION: NEED TO MAKE PRETTY
def ExtractBBoxFromImg(source_image, BBOX_LEFT, BBOX_TOP, BBOX_WIDTH, BBOX_HEIGHT):
  height, width, channels = source_image.shape
  start_row, start_col = int(width * BBOX_LEFT), int(height * BBOX_TOP) 
  end_row, end_col = start_row+int(width * BBOX_WIDTH), start_col+int(height * BBOX_HEIGHT)
  crop_img = source_image[start_col:start_col+int(height * BBOX_HEIGHT),start_row:start_row+int(width * BBOX_WIDTH)]
  return crop_img

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
  

In [10]:
df_tagged_data.head()

,_c0,_c1,_c2
0,000001,[],/9j/4RzHRXhpZgAASUkqAAgAAAARABABAgAgAAAA2gAAAC...
1,000002,[],/9j/4AAQSkZJRgABAQAASABIAAD/4QBMRXhpZgAATU0AKg...
2,000003,[],/9j/4AAQSkZJRgABAQAASABIAAD/4QBYRXhpZgAATU0AKg...
3,000004,[],/9j/4AAQSkZJRgABAQAASABIAAD/4QBMRXhpZgAATU0AKg...
4,000005,[],/9j/4AAQSkZJRgABAQAASABIAAD/4QBMRXhpZgAATU0AKg...


In [11]:
tag_counter = dict()
item_counter = dict()
image_counter = dict()
output_dir = ''
exception_counter = 0

for index, row in df_tagged_data.iterrows():
    imageTags = []
    taggedImageWithRegion = []
    output_dir = ''

    try:
        # Parse JSON Tag Data
        record_id = row['_c0']
        json_text = row['_c1']
        tag_data = json.loads(json_text)

        # Load Image Byte Data from File
        image_byte_data = binascii.a2b_base64(row['_c2'])
        nparr_image_byte_data = np.fromstring(image_byte_data, np.uint8)
        cv2_image = cv2.imdecode(nparr_image_byte_data, cv2.IMREAD_COLOR)

        for tag in tag_data:
            tag_class = ''
            tag_rect = ''
            output_dir = ''
            item_number = ''
            tag_class = tag["class"]
            tag_rect = tag["rect"]

            if (tag_class.lower() != 'product' and tag_class.lower() != 'skip' and tag_class.lower() != 'gap'):
                # Extract Item # from Existing Tag Name
                item_number = tag_class[tag_class.rfind('-') + 1:tag_class.find('_')]
                #print('Item #: {}'.format(item_number))
                tag_counter[tag_class] = tag_counter.get(tag_class, 0) + 1
                item_counter[item_number] = item_counter.get(item_number, 0) + 1
                
                #Image Save Path for this Item
                output_dir = os.path.join(IMAGE_OUTPUT_DIR, item_number)

                # Create folder for this item if it's the first time we see it
                if item_counter[item_number] == 1:
                    try:
                        print('Creating: {}'.format(output_dir))
                        # dbutils.fs.mkdirs(output_dir)
                        os.mkdir(output_dir)

                    except OSError as exc:
                        print('[ERROR] Exception Occurred on Record: {}'.format(record_id))
                        print(ex)
                        if exc.errno != errno.EEXIST:
                            raise
                            pass


                # Write contents of RECT.
                facing = ExtractBBoxFromImg(cv2_image, tag_rect[0], tag_rect[1], tag_rect[2], tag_rect[3])
                save_status = cv2.imwrite(output_dir + '/' + str(item_counter[item_number]) + '.jpg', facing)
                if save_status == False:
                  print('[ERROR] Error Occurred Saving Image: {}'.format(record_id))
                  print('[ERROR] Output Directory: {}'.format(output_dir))

    except Exception as ex:
        print('[ERROR] Exception Occurred on Record: {}'.format(record_id))
        print(ex)
        exception_counter = + 1
        continue

Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176815
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176743
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10129375
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176723
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1020294
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176726
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10202950
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10129362
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176814
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10129365
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10129397
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1012934
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176762
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197123
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10141492
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10141489
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10141493
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197122
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10141494
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197125
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197124
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197127
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176739
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10141490
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197121
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10129366
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197126
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10129373
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176763
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176755
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10197128
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176761
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176754
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10176807
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193241
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1019323
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193255
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193238
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1013724
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193239
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193235
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1019325
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193256
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10137237
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10137238
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193236
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10142529
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10142637
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10191686
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10191680
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10109268
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10191692
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10109272
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10192356
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10192394
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193263
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1019208
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/1011941
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10192353
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10191677
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10109275
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193259
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193257
Creating: /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/10193254
Creating: /local_disk0/tmp/PRODUCT_IMAG

In [12]:
%sh
###COMPRESS ALL FOLDERS & COPY TO DATA LAKE
pushd /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER/
zip -r /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER_DATA.ZIP .
cp /local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER_DATA.ZIP /dbfs/mnt/RAW/FILES/SYNAPSE/POC/UDACITY/PRODUCT_IMAGES_CLASSIFIER/PRODUCT_IMAGES_CLASSIFIER_DATA.ZIP
popd

/local_disk0/tmp/PRODUCT_IMAGES_CLASSIFIER /databricks/driver
 adding: 10119963/ (stored 0%)
 adding: 10119963/59.jpg (deflated 0%)
 adding: 10119963/95.jpg (deflated 1%)
 adding: 10119963/80.jpg (deflated 1%)
 adding: 10119963/54.jpg (deflated 2%)
 adding: 10119963/12.jpg (deflated 1%)
 adding: 10119963/8.jpg (deflated 1%)
 adding: 10119963/67.jpg (deflated 1%)
 adding: 10119963/16.jpg (deflated 2%)
 adding: 10119963/73.jpg (deflated 2%)
 adding: 10119963/72.jpg (deflated 3%)
 adding: 10119963/23.jpg (deflated 1%)
 adding: 10119963/55.jpg (deflated 2%)
 adding: 10119963/14.jpg (deflated 2%)
 adding: 10119963/88.jpg (deflated 2%)
 adding: 10119963/62.jpg (deflated 1%)
 adding: 10119963/22.jpg (deflated 1%)
 adding: 10119963/15.jpg (deflated 1%)
 adding: 10119963/79.jpg (deflated 1%)
 adding: 10119963/61.jpg (deflated 1%)
 adding: 10119963/60.jpg (deflated 1%)
 adding: 10119963/32.jpg (deflated 1%)
 adding: 10119963/18.jpg (deflated 1%)
 adding: 10119963/81.jpg (deflated 1%)
 adding: 10119963/27.jpg (deflated 1%)
 adding: 10119963/85.jpg (deflated 3%)
 adding: 10119963/84.jpg (deflated 2%)
 adding: 10119963/46.jpg (deflated 1%)
 adding: 10119963/53.jpg (deflated 1%)
 adding: 10119963/20.jpg (deflated 1%)
 adding: 10119963/49.jpg (deflated 2%)
 adding: 10119963/5.jpg (deflated 1%)
 adding: 10119963/25.jpg (deflated 1%)
 adding: 10119963/58.jpg (deflated 1%)
 adding: 10119963/21.jpg (deflated 1%)
 adding: 10119963/64.jpg (deflated 1%)
 adding: 10119963/41.jpg (deflated 2%)
 adding: 10119963/33.jpg (deflated 1%)
 adding: 10119963/45.jpg (deflated 1%)
 adding: 10119963/78.jpg (deflated 1%)
 adding: 10119963/87.jpg (deflated 3%)
 adding: 10119963/50.jpg (deflated 1%)
 adding: 10119963/43.jpg (deflated 2%)
 adding: 10119963/100.jpg (deflated 1%)
 adding: 10119963/40.jpg (deflated 2%)
 adding: 10119963/44.jpg (deflated 2%)
 adding: 10119963/26.jpg (deflated 1%)
 adding: 10119963/38.jpg (deflated 1%)
 adding: 10119963/94.jpg (deflated 1%)
 adding: 10119963/24.jpg (deflated 1%)
 adding: 10119963/93.jpg (deflated 1%)
 adding: 10119963/30.jpg (deflated 1%)
 adding: 10119963/31.jpg (deflated 1%)
 adding: 10119963/48.jpg (deflated 2%)
 adding: 10119963/74.jpg (deflated 1%)
 adding: 10119963/98.jpg (deflated 1%)
 adding: 10119963/102.jpg (deflated 1%)
 adding: 10119963/63.jpg (deflated 1%)
 adding: 10119963/35.jpg (deflated 1%)
 adding: 10119963/68.jpg (deflated 1%)
 adding: 10119963/77.jpg (deflated 1%)
 adding: 10119963/69.jpg (deflated 1%)
 adding: 10119963/10.jpg (deflated 2%)
 adding: 10119963/29.jpg (deflated 1%)
 adding: 10119963/39.jpg (deflated 1%)
 adding: 10119963/91.jpg (deflated 1%)
 adding: 10119963/47.jpg (deflated 2%)
 adding: 10119963/9.jpg (deflated 2%)
 adding: 10119963/52.jpg (deflated 1%)
 adding: 10119963/71.jpg (deflated 2%)
 adding: 10119963/75.jpg (deflated 1%)
 adding: 10119963/7.jpg (deflated 1%)
 adding: 10119963/76.jpg (deflated 1%)
 adding: 10119963/96.jpg (deflated 0%)
 adding: 10119963/51.jpg (deflated 1%)
 adding: 10119963/34.jpg (deflated 1%)
 adding: 10119963/92.jpg (deflated 1%)
 adding: 10119963/37.jpg (deflated 1%)
 adding: 10119963/83.jpg (deflated 2%)
 adding: 10119963/19.jpg (deflated 1%)
 adding: 10119963/28.jpg (deflated 1%)
 adding: 10119963/36.jpg (deflated 1%)
 adding: 10119963/65.jpg (deflated 1%)
 adding: 10119963/1.jpg (deflated 1%)
 adding: 10119963/2.jpg (deflated 1%)
 adding: 10119963/82.jpg (deflated 1%)
 adding: 10119963/42.jpg (deflated 2%)
 adding: 10119963/101.jpg (deflated 1%)
 adding: 10119963/57.jpg (deflated 1%)
 adding: 10119963/86.jpg (deflated 2%)
 adding: 10119963/3.jpg (deflated 1%)
 adding: 10119963/97.jpg (deflated 1%)
 adding: 10119963/66.jpg (deflated 0%)
 adding: 10119963/4.jpg (deflated 1%)
 adding: 10119963/17.jpg (deflated 1%)
 adding: 10119963/89.jpg (deflated 1%)
 adding: 10119963/70.jpg (deflated 1%)
 adding: 10119963/13.jpg (deflated 2%)
 adding: 10119963/11.jpg (deflated 2%)
 adding: 10119963/56.jpg (deflated 1%)
 adding: 10119963/6.jpg (deflated 1%)
 adding: 1011

In [13]:
#Display the # of Images by Item
for item_number, image_count in sorted(item_counter.items(), key=lambda item: item[1], reverse=True):
  print('{}: {}'.format(item_number, image_count))

10192088: 2301
10191677: 1656
10121869: 1311
10197123: 1271
10191686: 1192
10197125: 1051
10191692: 1029
10109268: 926
10191680: 861
10121870: 763
10121871: 715
10121868: 708
10192353: 688
10121872: 591
10108664: 587
10109264: 573
10192356: 568
10109272: 545
10109266: 540
10192091: 536
10197124: 507
10141489: 480
10109275: 466
10172361: 463
10192085: 461
10193241: 445
10141490: 445
10121874: 432
10192375: 425
10192365: 421
10172358: 418
10172367: 403
10192359: 370
10119951: 370
10193239: 361
10197122: 339
10176726: 324
10193238: 305
10197126: 282
10141494: 267
10197127: 258
10176807: 255
10192390: 244
10197128: 229
10118456: 228
10176723: 217
10193253: 216
10193236: 213
10142529: 208
10142637: 201
10141493: 196
10197121: 195
10142635: 186
10141492: 185
10142531: 176
10129343: 164
1019239: 161
10119956: 157
10176739: 156
10142633: 149
10129365: 148
10193240: 148
10193256: 148
10142631: 146
10137237: 143
10193235: 142
10176763: 135
10119954: 135
10193255: 133
10120054: 133
10202946: 128
10137240: 124
10176755: 122
10176743: 121
10176727: 120
10142638: 117
1019713: 114
10176814: 114
10159239: 103
10119963: 102
10129397: 98
10129375: 96
10137238: 96
10129393: 94
10119961: 94
10120052: 92
10176762: 91
10142542: 89
10193254: 86
10193237: 84
10142643: 76
10176753: 72
10119418: 71
10176815: 70
10129345: 67
10202950: 67
10192154: 64
10129373: 63
10193260: 63
10168005: 54
10192394: 52
10129362: 52
10129366: 51
10167964: 50
10193264: 48
10202944: 45
10197129: 43
10191689: 42
10168008: 40
10193259: 40
10118457: 39
1019208: 32
10192398: 31
10176761: 30
10129379: 29
10119081: 23
10167996: 23
10176754: 23
10176818: 22
10168002: 21
1019237: 19
10193265: 10
10167999: 9
10191326: 9
10191439: 8
10193257: 8
1011941: 8
10176817: 7
10191333: 7
10191328: 7
10191445: 7
1013724: 7
10167970: 7
10191324: 6
10191422: 6
10191433: 6
1019712: 6
1019209: 5
10191435: 5
10191335: 5
10176756: 5
10176725: 4
1020294: 4
10118455: 4
10191216: 4
10191215: 3
10137226: 3
1019236: 2
10193263: 2
1014263: 2
10192396: 2
1012934: 2
10191217: 2
1019323: 2
: 1
10129376: 1
10129349: 1
10176759: 1
1019325: 1
WHISKA: 1
10197131: 1
10176760: 1